**Links:**
* Wikipedia: https://en.wikipedia.org/wiki/Boggle
* How to play? https://www.wikihow.com/Play-Boggle
* English dice configuration: https://boardgames.stackexchange.com/questions/29264/boggle-what-is-the-dice-configuration-for-boggle-in-various-languages
* More configurations: https://en.wikipedia.org/wiki/Talk%3aBoggle#Boggle_Variants
* Inspiration for solver [1]: https://medium.com/@ashalabi13/solving-boggle-using-depth-first-searches-and-prefix-trees-9c3faa89ea99
* [2] https://www.geeksforgeeks.org/boggle-find-possible-words-board-characters/
* PyDictionary: https://pypi.org/project/english-words/
* English words list: https://github.com/dwyl/english-words
* Estonian word lists (based on dictionaries): https://www.eki.ee/tarkvara/wordlist/

In [117]:
import numpy as np
import json
import time

In [84]:
# Find every possible affix with a length of 2
def find_affixes(dictionary):
    affixes = dict()
    for word in dictionary.keys():
        for n in range(len(word)):
            if n < len(word)-1:
                affixes[word[n]+word[n+1]] = 1
    return affixes

In [128]:
def check_chars(word):
    # Remove words containing these characters from the dictionary
    chars = ['ž','c','z','y','w','q','x','é','è','ê','α']
    for char in word.lower():
        if char in chars:
            return False
    if word.isalpha():
        return True

In [129]:
def get_dictionary(filename):
    if '.txt' in filename:
        with open(filename, 'r', encoding='utf-8') as f:
            words_list = f.read().splitlines()
        words = {word.lower():1 for word in words_list if check_chars(word)}
    elif '.json' in filename:
        with open(filename) as json_file:
            words_raw = json.load(json_file)
        words = {word:1 for word in words_raw if word.isalpha()}
            
    # If word has a length of 3, add a value to prefix tree to indicate it
    # Create a prefix tree
    prefix_tree = dict()
    for k in list(words.keys()):
        # Prefix is also a word
        if len(k) == 3:
            prefix_tree[k] = 1
        # Prefix itself isn't a word
        elif len(k) > 3:
            prefix_tree[k[0:3]] = 0
        # Remove words smaller than length of 3
        elif len(k) < 3:
            del words[k]
    return words, prefix_tree

In [36]:
# Create a random board with given dice
def create_board(dice):
    unused_dice = dice.copy()
    board = [list(zeros) for zeros in np.zeros((4,4))]
    for i in range(4):
        for j in range(4):
            # Take a random die from the set of dice
            die = unused_dice.pop(np.random.randint(len(unused_dice)))
            # Assign a random letter from the die onto the board
            board[i][j] = die[np.random.randint(6)]
    return board

In [135]:
# Get adjacent vertices for given vertex
def get_adj(i,j):
    adj_indices = [(i-1,j), (i+1,j), (i,j-1), (i,j+1), (i-1,j-1), (i+1,j-1), (i-1,j+1), (i+1,j+1)]
    return list(filter(lambda x : x[0] in range(0,4) and x[1] in range(0,4), adj_indices))

# For quick access save adjacent vertices into a map
adj_map = dict()
for i in range(4):
    for j in range(4):
        adj_map[(i,j)] = get_adj(i,j)

In [116]:
# Depth-First Search from lecture slides
def DFS(vertices, edges, prefix_tree, affix_tree, dictionary):
    un_visited_vertices = [[False for j in range(4)] for i in range(4)]
    #un_visited_vertices = [(i,j) for i in range(4) for j in range(4)]
    for i in range(4):
        for j in range(4):
            string = ""#vertices[i][j]
            visited_vertices = [i[:] for i in un_visited_vertices]
            #print("starting from", vertices[i][j], "at", (i,j))
            DFS_visit((i,j), vertices, edges, visited_vertices, string, prefix_tree, affix_tree, dictionary)
            
def DFS_visit(vertice, vertices, edges, visited_vertices, string, prefix_tree, affix_tree, dictionary):
    #global comparisons
    #global all_comparisons
    string += vertices[vertice[0]][vertice[1]]
    #print(string.lower())
    visited_vertices[vertice[0]][vertice[1]] = True
    if affix_tree != None:
        if len(string) > 1:
            affix = string[len(string)-2].lower()+string[len(string)-1].lower()
            #print(affix)
            if affix_tree.get(affix) == None:
                return
    else:
        if len(string) == 3:
            if prefix_tree.get(string.lower()) == None:
                return
    if dictionary.get(string.lower()) != None:
        print(string)
    # Find adjacent vertices
    adj = edges[vertice]
    for adj_v in adj:
        if visited_vertices[adj_v[0]][adj_v[1]] == False:
            branch_visited_vertices = [i[:] for i in visited_vertices]
            DFS_visit(adj_v, vertices, edges, branch_visited_vertices, string, prefix_tree, affix_tree, dictionary)
    # Backtrack
    string = string.rstrip(string[-1])
    visited_vertices[vertice[0]][vertice[1]] = False

In [132]:
# en: https://boardgames.stackexchange.com/questions/29264/boggle-what-is-the-dice-configuration-for-boggle-in-various-languages
dice = [["R", "I", "F", "O", "B", "X"],
       ["I", "F", "E", "H", "E", "Y"],
       ["D", "E", "N", "O", "W", "S"],
       ["U", "T", "O", "K", "N", "D"],
       ["H", "M", "S", "R", "A", "O"],
       ["L", "U", "P", "E", "T", "S"],
       ["A", "C", "I", "T", "O", "A"],
       ["Y", "L", "G", "K", "U", "E"],
       ["Qu", "B", "M", "J", "O", "A"],
       ["E", "H", "I", "S", "P", "N"],
       ["V", "E", "T", "I", "G", "N"],
       ["B", "A", "L", "I", "Y", "T"],
       ["E", "Z", "A", "V", "N", "D"],
       ["R", "A", "L", "E", "S", "C"],
       ["U", "W", "I", "L", "R", "G"],
       ["P", "A", "C", "E", "M", "D"]]

In [108]:
### TEST BLOCK ###

dictionary = {"test":"", "boggle":"", "his":"", "josh":"", "toe":"", "joe":"", "she":""}
prefix_tree = {"tes": "test", "bog": "boggle", "his": "his", "jos": "josh", "toe": "toe", "joe": "joe", "she":"she"}
affix_tree = find_affixes(dictionary)
#affix_tree = None
board = [['T', 'E', 'S', 'T'],
         ['B', 'O', 'H', 'L'],
         ['G', 'J', 'I', 'H'],
         ['G', 'L', 'E', 'S']]
found_words = DFS(board, adj_map, prefix_tree, affix_tree, dictionary)
board

### TEST BLOCK ###

TEST
TOE
SHE
BOGGLE
HIS
JOE
JOSH
HIS
SHE


[['T', 'E', 'S', 'T'],
 ['B', 'O', 'H', 'L'],
 ['G', 'J', 'I', 'H'],
 ['G', 'L', 'E', 'S']]

In [109]:
en_dict, prefix_tree = get_dictionary('words_dictionary.json')

In [110]:
affix_tree = find_affixes(en_dict)

In [112]:
print("Dictionary length:", len(en_dict), ", prefix tree length:", len(prefix_tree), ", affix tree length:", len(affix_tree))

Dictionary length: 369648 , prefix tree length: 4799 , affix tree length: 654


In [133]:
board = create_board(dice)

In [123]:
start_time = time.time()
found_words = DFS(board, adj_map, prefix_tree, affix_tree, en_dict)
end_time = time.time()
print(end_time-start_time, "s")

XED
XENYL
KELD
KED
KEA
KEX
KEN
KEND
KENYA
KENYA
KEA
KEAS
KHA
KHALSA
KAAS
KAE
KAE
KAL
KALE
KALEND
KALA
KALA
KALAN
KAE
KAES
HAE
HAED
HAEN
HAE
HAK
HAKE
HAKEA
HAL
HALE
HALED
HALA
HALAS
HALA
HALS
HALSE
HAO
HAE
HAES
HOE
HEL
HELAS
HELD
HED
HEAL
HEALD
HEALS
HEX
HEN
HEND
HENDLY
HEAL
HEALD
HEALS
HEO
HEE
HEAL
HEALED
HEALD
HEALS
HEAL
HEALED
HEALD
HEALS
OHED
OAK
OAKEN
DEL
DELY
DELAY
DELAYS
DELAY
DELAYS
DELS
DEA
DEAL
DEALS
DEX
DEN
DENAY
DENY
DEA
DEAS
DEAL
DEALS
ELYSEE
ELA
ELD
ELA
ELAN
ELAND
ELA
ELS
ELSA
ELSE
EDNA
END
ENL
ENLAY
ENLAY
EASY
EASE
AHO
AAS
AAL
AALS
AKE
AKELA
AKELAS
AKELA
AKED
ALE
ALEX
ALEN
ALEA
ALY
ALYA
ALYA
ALN
ALA
ALAS
ALAE
ALAE
ALAY
ALAE
ALD
ALDEN
ALDEA
ALA
ALAN
ALAND
ALANE
ALAY
ALS
ALSO
AES
EASY
EASE
NAY
NAYS
NALE
NALED
NEK
NED
NEA
NEAL
NEA
NEAL
NYALA
NYALAS
NYALA
NYAS
NYALA
NYALA
LEK
LEKHA
LED
LEA
LEAH
LEAK
LEX
LEN
LEND
LENA
LEA
LEAS
LEASE
LYS
LYSE
LYN
LYAS
LYASE
LAS
LASE
LAEN
LAY
LAYS
LAYNE
LAO
LAOS
LAN
LAND
LANDE
LANE
LAY
LAYS
LAYNE
LAH
LAEN
LAK
LAKE
LAKED
LAKH
LAO
AAH
AAHED
AAL
AA

In [122]:
start_time = time.time()
found_words = DFS(board, adj_map, prefix_tree, None, en_dict)
end_time = time.time()
print(end_time-start_time, "s")

XED
XENYL
KELD
KED
KEA
KEX
KEN
KEND
KENYA
KENYA
KEA
KEAS
KHA
KHALSA
KAAS
KAE
KAE
KAL
KALE
KALEND
KALA
KALA
KALAN
KAE
KAES
HAE
HAED
HAEN
HAE
HAK
HAKE
HAKEA
HAL
HALE
HALED
HALA
HALAS
HALA
HALS
HALSE
HAO
HAE
HAES
HOE
HEL
HELAS
HELD
HED
HEAL
HEALD
HEALS
HEX
HEN
HEND
HENDLY
HEAL
HEALD
HEALS
HEO
HEE
HEAL
HEALED
HEALD
HEALS
HEAL
HEALED
HEALD
HEALS
OHED
OAK
OAKEN
DEL
DELY
DELAY
DELAYS
DELAY
DELAYS
DELS
DEA
DEAL
DEALS
DEX
DEN
DENAY
DENY
DEA
DEAS
DEAL
DEALS
ELYSEE
ELA
ELD
ELA
ELAN
ELAND
ELA
ELS
ELSA
ELSE
EDNA
END
ENL
ENLAY
ENLAY
EASY
EASE
AHO
AAS
AAL
AALS
AKE
AKELA
AKELAS
AKELA
AKED
ALE
ALEX
ALEN
ALEA
ALY
ALYA
ALYA
ALN
ALA
ALAS
ALAE
ALAE
ALAY
ALAE
ALD
ALDEN
ALDEA
ALA
ALAN
ALAND
ALANE
ALAY
ALS
ALSO
AES
EASY
EASE
NAY
NAYS
NALE
NALED
NEK
NED
NEA
NEAL
NEA
NEAL
NYALA
NYALAS
NYALA
NYAS
NYALA
NYALA
LEK
LEKHA
LED
LEA
LEAH
LEAK
LEX
LEN
LEND
LENA
LEA
LEAS
LEASE
LYS
LYSE
LYN
LYAS
LYASE
LAS
LASE
LAEN
LAY
LAYS
LAYNE
LAO
LAOS
LAN
LAND
LANDE
LANE
LAY
LAYS
LAYNE
LAH
LAEN
LAK
LAKE
LAKED
LAKH
LAO
AAH
AAHED
AAL
AA

In [124]:
et_dice = [['l', 'i', 'õ', 's', 'j', 'f'],
           ['i', 'õ', 'a', 'd', 'a', 'g'],
           ['p', 'a', 't', 's', 'b', 'u'],
           ['m', 'k', 's', 'ü', 't', 'p'],
           ['d', 'v', 'u', 'l', 'e', 's'],
           ['n', 'm', 'o', 'a', 'k', 'u'],
           ['e', 'r', 'i', 'k', 's', 'e'],
           ['g', 'n', 'h', 'ü', 'm', 'a'],
           ['š', 'j', 'v', 'J', 's', 'e'],
           ['a', 'd', 'i', 'u', 'o', 't'],
           ['ä', 'a', 'k', 'i', 'h', 't'],
           ['j', 'e', 'n', 'i', 'g', 'k'],
           ['a', 'ö', 'e', 'ä', 't', 'p'],
           ['l', 'e', 'n', 'a', 'u', 'r'],
           ['m', 'b', 'i', 'n', 'l', 'h'],
           ['o', 'e', 'r', 'a', 'v', 'p']]

In [130]:
et_dict, et_prefix_tree = get_dictionary('lemmad2013.txt')
et_affix_tree = find_affixes(et_dict)

In [134]:
et_board = create_board(et_dice)

In [139]:
start_time = time.time()
found_words = DFS(et_board, adj_map, et_prefix_tree, et_affix_tree, et_dict)
end_time = time.time()
print(end_time-start_time, "s")

uig
uig
uih
viki
viki
viig
viigi
viik
viiki
viu
viht
vihk
vihik
vihk
vuih
viu
viik
viiki
viig
viigi
viir
viki
viki
rivi
rivik
riiu
riigi
riiv
riik
riht
rihti
rihk
rihk
igi
igi
igi
igi
irv
irvi
irvik
ihk
ihk
lik
liht
lihk
lihk
kivi
kii
kiiv
kih
kiht
kihi
kihik
kihl
kihk
kii
kiiv
kiir
kivi
hiv
hii
hiiu
higi
higi
hirv
ihk
ihii
ihk
itk
itk
igi
igi
pigi
pigi
piki
piki
pikt
ptk
ptk
tihk
tihi
tihk
kih
kiht
kihk
kihi
kihik
kihl
13.578789234161377 s


In [140]:
start_time = time.time()
found_words = DFS(et_board, adj_map, et_prefix_tree, None, et_dict)
end_time = time.time()
print(end_time-start_time, "s")

uig
uig
uih
viki
viki
viig
viigi
viik
viiki
viu
viht
vihk
vihik
vihk
vuih
viu
viik
viiki
viig
viigi
viir
viki
viki
rivi
rivik
riiu
riigi
riiv
riik
riht
rihti
rihk
rihk
igi
igi
igi
igi
irv
irvi
irvik
ihk
ihk
lik
liht
lihk
lihk
kivi
kii
kiiv
kih
kiht
kihi
kihik
kihl
kihk
kii
kiiv
kiir
kivi
hiv
hii
hiiu
higi
higi
hirv
ihk
ihii
ihk
itk
itk
igi
igi
pigi
pigi
piki
piki
pikt
ptk
ptk
tihk
tihi
tihk
kih
kiht
kihk
kihi
kihik
kihl
8.363850355148315 s


In [142]:
len(et_affix_tree)

587